In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

In [3]:
all_files = glob.glob("../data/raw/311/*.csv")
df_list = []
for file in sorted(all_files):
    df_temp = pd.read_csv(file)
    df_list.append(df_temp)

df_311 = pd.concat(df_list, ignore_index=True)

df_311["created_date"] = pd.to_datetime(df_311["created_date"])
df_311["date"] = df_311["created_date"].dt.date

print(f"Loaded {len(df_311):,} 311 records")
df_311.head()

Loaded 6,900,519 311 records


,unique_key,created_date,closed_date,agency,complaint_type,descriptor,status,borough,latitude,longitude,date
0,56416396,2023-01-01 00:00:00,2023-01-03T14:56:48.000,DOHMH,Food Poisoning,1 or 2,Closed,QUEENS,40.727630,-73.873614,2023-01-01
1,56417527,2023-01-01 00:00:09,2023-01-01T00:36:06.000,NYPD,Illegal Fireworks,NaN,Closed,BROOKLYN,40.640915,-73.973642,2023-01-01
2,56416252,2023-01-01 00:00:42,2023-01-01T17:34:15.000,NYPD,Noise - Residential,Loud Music/Party,Closed,BRONX,40.853848,-73.917709,2023-01-01
3,56418795,2023-01-01 00:00:45,2023-01-01T01:24:10.000,NYPD,Illegal Parking,Posted Parking Sign Violation,Closed,MANHATTAN,40.754875,-74.000747,2023-01-01
4,56418136,2023-01-01 00:00:46,2023-01-01T01:01:43.000,NYPD,Noise - Residential,Loud Music/Party,Closed,BROOKLYN,40.620665,-73.926040,2023-01-01


In [4]:
top_complaint_types = df_311["complaint_type"].value_counts().head(15).index.tolist()
print("Top 15 complaint types:")
print(top_complaint_types)

complaint_by_type = (
    df_311[df_311["complaint_type"].isin(top_complaint_types)]
    .groupby(["date", "complaint_type"])
    .size()
    .unstack(fill_value=0)
)

complaint_by_type.columns = [
    "complaints_" + col.lower().replace("/", "_").replace(" ", "_").replace("-", "_")
    for col in complaint_by_type.columns
]

complaint_by_type = complaint_by_type.reset_index()
print(f"\nComplaint type features shape: {complaint_by_type.shape}")
complaint_by_type.head()

Top 15 complaint types:
['Illegal Parking', 'Noise - Residential', 'HEAT/HOT WATER', 'Blocked Driveway', 'Noise - Street/Sidewalk', 'UNSANITARY CONDITION', 'Noise - Commercial', 'Street Condition', 'Abandoned Vehicle', 'PLUMBING', 'PAINT/PLASTER', 'Water System', 'Dirty Condition', 'Noise', 'Derelict Vehicles']

Complaint type features shape: (731, 16)


,date,complaints_abandoned_vehicle,complaints_blocked_driveway,complaints_derelict_vehicles,complaints_dirty_condition,complaints_heat_hot_water,complaints_illegal_parking,complaints_noise,complaints_noise___commercial,complaints_noise___residential,complaints_noise___street_sidewalk,complaints_paint_plaster,complaints_plumbing,complaints_street_condition,complaints_unsanitary_condition,complaints_water_system
0,2023-01-01,71,489,58,58,466,1024,39,194,1400,418,37,42,74,98,95
1,2023-01-02,165,371,135,117,681,1092,57,81,679,245,99,150,154,225,121
2,2023-01-03,167,433,133,148,688,1234,158,73,543,117,204,223,276,376,152
3,2023-01-04,199,439,157,160,518,1358,143,64,443,173,233,255,271,402,200
4,2023-01-05,219,455,170,149,809,1278,163,103,585,187,172,185,267,341,184


In [5]:
borough_counts = df_311.groupby(["date", "borough"]).size().unstack(fill_value=0)
borough_counts.columns = ["complaints_" + col.lower() for col in borough_counts.columns]
borough_counts = borough_counts.reset_index()

print(f"Borough features shape: {borough_counts.shape}")
borough_counts.head()

Borough features shape: (731, 7)


,date,complaints_bronx,complaints_brooklyn,complaints_manhattan,complaints_queens,complaints_staten island,complaints_unspecified
0,2023-01-01,1202,1838,1253,1686,228,2
1,2023-01-02,1370,1927,1468,1567,236,112
2,2023-01-03,1541,2700,1562,2002,443,7
3,2023-01-04,1675,2701,1629,2037,412,8
4,2023-01-05,1642,2714,1685,1959,404,10


In [6]:
daily_total = df_311.groupby("date").size().reset_index(name="total_complaints")
print(f"Daily total shape: {daily_total.shape}")
daily_total.head()

Daily total shape: (731, 2)


,date,total_complaints
0,2023-01-01,6209
1,2023-01-02,6680
2,2023-01-03,8255
3,2023-01-04,8462
4,2023-01-05,8414


In [7]:
df_311_features = daily_total.merge(complaint_by_type, on="date", how="left")
df_311_features = df_311_features.merge(borough_counts, on="date", how="left")

print(f"Combined 311 features shape: {df_311_features.shape}")
print(f"Columns: {df_311_features.columns.tolist()}")
df_311_features.head()

Combined 311 features shape: (731, 23)
Columns: ['date', 'total_complaints', 'complaints_abandoned_vehicle', 'complaints_blocked_driveway', 'complaints_derelict_vehicles', 'complaints_dirty_condition', 'complaints_heat_hot_water', 'complaints_illegal_parking', 'complaints_noise', 'complaints_noise___commercial', 'complaints_noise___residential', 'complaints_noise___street_sidewalk', 'complaints_paint_plaster', 'complaints_plumbing', 'complaints_street_condition', 'complaints_unsanitary_condition', 'complaints_water_system', 'complaints_bronx', 'complaints_brooklyn', 'complaints_manhattan', 'complaints_queens', 'complaints_staten island', 'complaints_unspecified']


,date,total_complaints,complaints_abandoned_vehicle,complaints_blocked_driveway,complaints_derelict_vehicles,complaints_dirty_condition,complaints_heat_hot_water,complaints_illegal_parking,complaints_noise,complaints_noise___commercial,...,complaints_plumbing,complaints_street_condition,complaints_unsanitary_condition,complaints_water_system,complaints_bronx,complaints_brooklyn,complaints_manhattan,complaints_queens,complaints_staten island,complaints_unspecified
0,2023-01-01,6209,71,489,58,58,466,1024,39,194,...,42,74,98,95,1202,1838,1253,1686,228,2
1,2023-01-02,6680,165,371,135,117,681,1092,57,81,...,150,154,225,121,1370,1927,1468,1567,236,112
2,2023-01-03,8255,167,433,133,148,688,1234,158,73,...,223,276,376,152,1541,2700,1562,2002,443,7
3,2023-01-04,8462,199,439,157,160,518,1358,143,64,...,255,271,402,200,1675,2701,1629,2037,412,8
4,2023-01-05,8414,219,455,170,149,809,1278,163,103,...,185,267,341,184,1642,2714,1685,1959,404,10


In [8]:
df_weather = pd.read_csv("../data/raw/weather/weather_nyc_2023-01-01_to_2024-12-31.csv")
df_weather["date"] = pd.to_datetime(df_weather["date"]).dt.date

print(f"Weather data shape: {df_weather.shape}")
df_weather.head()

Weather data shape: (731, 9)


,date,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,windspeed_10m_max,weathercode
0,2023-01-01,8.785417,11.95,4.95,1.1,1.1,0.0,17.399586,55.0
1,2023-01-02,7.981249,12.85,4.55,0.6,0.6,0.0,9.000000,51.0
2,2023-01-03,9.691669,12.85,6.30,9.4,9.4,0.0,19.373219,63.0
3,2023-01-04,13.081250,18.00,9.55,2.3,2.3,0.0,18.218275,53.0
4,2023-01-05,9.912500,13.15,7.45,0.0,0.0,0.0,12.074766,3.0


In [10]:
df = df_311_features.merge(df_weather, on="date", how="inner")

df["date"] = pd.to_datetime(df["date"])

# Time features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day_of_week"] = df["date"].dt.dayofweek  # Monday=0, Sunday=6
df["day_of_month"] = df["date"].dt.day
df["week_of_year"] = df["date"].dt.isocalendar().week
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)


# Season
def get_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    else:
        return "fall"


df["season"] = df["month"].apply(get_season)

df[["date", "year", "month", "day_of_week", "is_weekend", "season"]].head()

,date,year,month,day_of_week,is_weekend,season
0,2023-01-01,2023,1,6,1,winter
1,2023-01-02,2023,1,0,0,winter
2,2023-01-03,2023,1,1,0,winter
3,2023-01-04,2023,1,2,0,winter
4,2023-01-05,2023,1,3,0,winter


In [12]:
# Weather category features
df["is_rainy"] = (df["rain_sum"] > 0).astype(int)
df["is_snowy"] = (df["snowfall_sum"] > 0).astype(int)
df["is_precipitation"] = (df["precipitation_sum"] > 0).astype(int)

# Temperature categories
df["is_hot"] = (df["temperature_2m_mean"] > 25).astype(int)  # > 25°C
df["is_cold"] = (df["temperature_2m_mean"] < 0).astype(int)  # < 0°C
df["is_freezing"] = (df["temperature_2m_min"] < 0).astype(int)

# Temperature bins
df["temp_bin"] = pd.cut(
    df["temperature_2m_mean"],
    bins=[-np.inf, 0, 10, 20, np.inf],
    labels=["freezing", "cold", "mild", "hot"],
)

# Precipitation intensity
df["precip_intensity"] = pd.cut(
    df["precipitation_sum"],
    bins=[-np.inf, 0, 5, 15, np.inf],
    labels=["none", "light", "moderate", "heavy"],
)

df[["date", "is_rainy", "is_hot", "is_cold", "temp_bin", "precip_intensity"]].head(10)

,date,is_rainy,is_hot,is_cold,temp_bin,precip_intensity
0,2023-01-01,1,0,0,cold,light
1,2023-01-02,1,0,0,cold,light
2,2023-01-03,1,0,0,cold,moderate
3,2023-01-04,1,0,0,mild,light
4,2023-01-05,0,0,0,cold,none
5,2023-01-06,1,0,0,cold,moderate
6,2023-01-07,0,0,0,cold,none
7,2023-01-08,0,0,0,cold,none
8,2023-01-09,1,0,0,cold,light
9,2023-01-10,0,0,0,cold,none


In [13]:
# Lagged features (previous day's weather)
df = df.sort_values("date").reset_index(drop=True)

df["temp_prev_day"] = df["temperature_2m_mean"].shift(1)
df["precip_prev_day"] = df["precipitation_sum"].shift(1)
df["complaints_prev_day"] = df["total_complaints"].shift(1)

# Rolling averages (7-day window)
df["temp_rolling_7d"] = df["temperature_2m_mean"].rolling(window=7, min_periods=1).mean()
df["complaints_rolling_7d"] = df["total_complaints"].rolling(window=7, min_periods=1).mean()

df[["date", "temperature_2m_mean", "temp_prev_day", "temp_rolling_7d"]].head(10)

,date,temperature_2m_mean,temp_prev_day,temp_rolling_7d
0,2023-01-01,8.785417,NaN,8.785417
1,2023-01-02,7.981249,8.785417,8.383333
2,2023-01-03,9.691669,7.981249,8.819445
3,2023-01-04,13.081250,9.691669,9.884896
4,2023-01-05,9.912500,13.081250,9.890417
5,2023-01-06,5.993750,9.912500,9.240972
6,2023-01-07,3.683333,5.993750,8.447024
7,2023-01-08,1.339583,3.683333,7.383334
8,2023-01-09,3.091667,1.339583,6.684822
9,2023-01-10,2.675000,3.091667,5.682440


In [14]:
print(f"Shape: {df.shape}")
print(f"\nColumns ({len(df.columns)}):")
print(df.columns.tolist())
print(f"\nMissing values:")
print(df.isnull().sum()[df.isnull().sum() > 0])
print(f"\nData types:")
print(df.dtypes.value_counts())

Shape: (731, 51)

Columns (51):
['date', 'total_complaints', 'complaints_abandoned_vehicle', 'complaints_blocked_driveway', 'complaints_derelict_vehicles', 'complaints_dirty_condition', 'complaints_heat_hot_water', 'complaints_illegal_parking', 'complaints_noise', 'complaints_noise___commercial', 'complaints_noise___residential', 'complaints_noise___street_sidewalk', 'complaints_paint_plaster', 'complaints_plumbing', 'complaints_street_condition', 'complaints_unsanitary_condition', 'complaints_water_system', 'complaints_bronx', 'complaints_brooklyn', 'complaints_manhattan', 'complaints_queens', 'complaints_staten island', 'complaints_unspecified', 'temperature_2m_mean', 'temperature_2m_max', 'temperature_2m_min', 'precipitation_sum', 'rain_sum', 'snowfall_sum', 'windspeed_10m_max', 'weathercode', 'year', 'month', 'day_of_week', 'day_of_month', 'week_of_year', 'is_weekend', 'season', 'is_rainy', 'is_snowy', 'is_precipitation', 'is_hot', 'is_cold', 'is_freezing', 'temp_bin', 'precip_inte

In [15]:
# Convert categorical columns to numeric for modeling
df_encoded = df.copy()
df_encoded = pd.get_dummies(
    df_encoded, columns=["season", "temp_bin", "precip_intensity"], drop_first=True
)

# Save
output_path = "../data/processed/311_weather_features.csv"
df_encoded.to_csv(output_path, index=False)
print(f"\nSaved feature-engineered dataset to: {output_path}")
print(f"Final shape: {df_encoded.shape}")


Saved feature-engineered dataset to: ../data/processed/311_weather_features.csv
Final shape: (731, 57)
